<a href="https://colab.research.google.com/github/santiagonajera/PlanificacionProduccion/blob/main/ABC_XYZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# URL del archivo Excel
url = 'https://github.com/santiagonajera/PlanificacionProduccion/raw/refs/heads/main/Forecast-2026.xlsx'

# Leer la hoja 'forecast' con la primera columna como índice (meses)
df_forecast = pd.read_excel(url, sheet_name='forecast', index_col=0)

# Sumar las ventas por ítem (suma a lo largo de las filas para cada columna)
sales_sum = df_forecast.sum(axis=0)

# Leer la hoja 'Precio_Costo' con la primera columna como índice (ítems)
df_prices = pd.read_excel(url, sheet_name='Precio_Costo', index_col=0)

# Convertir los precios de formato con coma a punto decimal y a float
df_prices['PRECIO'] = df_prices['PRECIO'].astype(str).str.replace(',', '.').astype(float)

# Calcular el ingreso (revenue) para cada ítem: ventas sumadas * precio
revenues = {}
for item in sales_sum.index:
    if item in df_prices.index:
        revenues[item] = sales_sum[item] * df_prices.loc[item, 'PRECIO']

# Convertir a DataFrame para facilitar el manejo
df_revenues = pd.DataFrame(list(revenues.items()), columns=['Item', 'Revenue'])
df_revenues = df_revenues.sort_values(by='Revenue', ascending=False).reset_index(drop=True)

# Calcular el ingreso total
total_revenue = df_revenues['Revenue'].sum()

# Calcular el porcentaje acumulativo
df_revenues['CumRevenue'] = df_revenues['Revenue'].cumsum()
df_revenues['CumPerc'] = df_revenues['CumRevenue'] / total_revenue

# Asignar clases ABC
def assign_class(cum_perc):
    if cum_perc <= 0.6:
        return 'A'
    elif cum_perc <= 0.8:
        return 'B'
    else:
        return 'C'

df_revenues['Class'] = df_revenues['CumPerc'].apply(assign_class)

# Imprimir los resultados
print("Clasificación ABC de los ítems:")
print(df_revenues[['Item', 'Revenue', 'CumPerc', 'Class']])

ValueError: Worksheet named 'forecast' not found

In [2]:
import pandas as pd

# URL del archivo Excel
url = 'https://github.com/santiagonajera/PlanificacionProduccion/raw/refs/heads/main/Forecast-2026.xlsx'

# Leer la hoja 'Forecast' con la primera columna como índice (meses)
df_forecast = pd.read_excel(url, sheet_name='Forecast', index_col=0)

# Sumar las ventas por ítem (suma a lo largo de las filas para cada columna)
sales_sum = df_forecast.sum(axis=0)

# Leer la hoja 'Precio_Costo' con la primera columna como índice (ítems)
df_prices = pd.read_excel(url, sheet_name='Precio_Costo', index_col=0)

# Convertir los precios de formato con coma a punto decimal y a float
df_prices['PRECIO'] = df_prices['PRECIO'].astype(str).str.replace(',', '.').astype(float)

# Calcular el ingreso (revenue) para cada ítem: ventas sumadas * precio
revenues = {}
for item in sales_sum.index:
    if item in df_prices.index:
        revenues[item] = sales_sum[item] * df_prices.loc[item, 'PRECIO']

# Convertir a DataFrame para facilitar el manejo
df_revenues = pd.DataFrame(list(revenues.items()), columns=['Item', 'Revenue'])
df_revenues = df_revenues.sort_values(by='Revenue', ascending=False).reset_index(drop=True)

# Calcular el porcentaje acumulativo
df_revenues['CumRevenue'] = df_revenues['Revenue'].cumsum()
df_revenues['CumPerc'] = df_revenues['CumRevenue'] / df_revenues['Revenue'].sum()

# Asignar clases ABC
def assign_class(cum_perc):
    if cum_perc <= 0.6:
        return 'A'
    elif cum_perc <= 0.8:
        return 'B'
    else:
        return 'C'

df_revenues['Class'] = df_revenues['CumPerc'].apply(assign_class)

# Imprimir los resultados
print("Clasificación ABC de los ítems:")
print(df_revenues[['Item', 'Revenue', 'CumPerc', 'Class']])

# Guardar los resultados en un archivo Excel
df_revenues.to_excel('ABC_Classification_Result.xlsx', index=False)
print("\nResultados guardados en 'ABC_Classification_Result.xlsx'")

Clasificación ABC de los ítems:
       Item    Revenue   CumPerc Class
0    Item_6  261897.23  0.048266     A
1   Item_39  232804.00  0.091170     A
2   Item_49  218780.48  0.131489     A
3    Item_7  218632.77  0.171781     A
4    Item_4  208327.68  0.210174     A
5    Item_5  202872.11  0.247562     A
6   Item_38  179744.20  0.280688     A
7   Item_27  173868.10  0.312730     A
8   Item_20  169890.00  0.344039     A
9   Item_13  169719.71  0.375317     A
10  Item_10  167416.86  0.406171     A
11  Item_22  166969.60  0.436942     A
12  Item_24  165343.68  0.467414     A
13   Item_1  150256.08  0.495105     A
14  Item_46  138867.32  0.520697     A
15   Item_2  130753.04  0.544794     A
16  Item_16  129734.80  0.568703     A
17  Item_50  127080.32  0.592123     A
18   Item_3  124169.24  0.615006     B
19  Item_14  121012.41  0.637308     B
20  Item_23  120146.04  0.659450     B
21   Item_9  117472.72  0.681099     B
22  Item_34  101098.56  0.699731     B
23  Item_47  100672.88  0.718284

In [3]:
import pandas as pd
import numpy as np

# URL del archivo Excel
url = 'https://github.com/santiagonajera/PlanificacionProduccion/raw/refs/heads/main/Forecast-2026.xlsx'

# Leer la hoja 'Ventas' con la primera columna como índice (meses)
df_sales = pd.read_excel(url, sheet_name='Ventas', index_col=0)

# Calcular el coeficiente de variación para cada ítem usando los últimos 16 datos
cvs = {}
for item in df_sales.columns:
    # Tomar los últimos 16 valores, ignorando NaN
    series = df_sales[item].dropna().tail(16)
    if len(series) > 0:
        mean = np.mean(series)
        std = np.std(series)
        cv = std / mean if mean != 0 else 0
        cvs[item] = cv

# Si no hay datos, salir
if not cvs:
    print("No hay datos disponibles para calcular los CV.")
else:
    # Calcular percentiles
    cv_values = list(cvs.values())
    p33 = np.percentile(cv_values, 33)
    p67 = np.percentile(cv_values, 67)

    # Crear DataFrame con CV
    df_cv = pd.DataFrame.from_dict(cvs, orient='index', columns=['CV'])
    df_cv = df_cv.sort_values('CV').reset_index().rename(columns={'index': 'Item'})

    # Función para asignar clase XYZ
    def assign_xyz(cv, p33, p67):
        if cv <= p33:
            return 'X'
        elif cv <= p67:
            return 'Y'
        else:
            return 'Z'

    df_cv['Class'] = df_cv['CV'].apply(lambda x: assign_xyz(x, p33, p67))

    # Imprimir los resultados
    print("Clasificación XYZ de los ítems:")
    print(df_cv[['Item', 'CV', 'Class']])

    # Guardar los resultados en un archivo Excel
    df_cv.to_excel('XYZ_Classification_Result.xlsx', index=False)
    print("\nResultados guardados en 'XYZ_Classification_Result.xlsx'")

Clasificación XYZ de los ítems:
       Item        CV Class
0   Item_21  0.064900     X
1   Item_41  0.076121     X
2   Item_44  0.079172     X
3    Item_6  0.079889     X
4   Item_29  0.086172     X
5   Item_45  0.087404     X
6   Item_34  0.088867     X
7   Item_39  0.089368     X
8    Item_2  0.090762     X
9   Item_10  0.091388     X
10   Item_9  0.092884     X
11   Item_1  0.094740     X
12  Item_24  0.095171     X
13   Item_3  0.095259     X
14   Item_4  0.097296     X
15  Item_46  0.097750     X
16  Item_20  0.101705     X
17  Item_38  0.102612     Y
18  Item_23  0.103816     Y
19  Item_27  0.105956     Y
20  Item_14  0.107330     Y
21   Item_7  0.107599     Y
22  Item_15  0.109554     Y
23  Item_40  0.109594     Y
24  Item_22  0.112707     Y
25  Item_50  0.113429     Y
26  Item_11  0.116752     Y
27  Item_43  0.117150     Y
28  Item_49  0.122104     Y
29  Item_32  0.128981     Y
30  Item_13  0.131436     Y
31  Item_25  0.134911     Y
32  Item_31  0.141187     Y
33  Item_36  0.1

In [4]:
import pandas as pd
import numpy as np
import uuid

# URL del archivo Excel
url = 'https://github.com/santiagonajera/PlanificacionProduccion/raw/refs/heads/main/Forecast-2026.xlsx'

# --- Clasificación ABC ---
# Leer la hoja 'Forecast' con la primera columna como índice (meses)
df_forecast = pd.read_excel(url, sheet_name='Forecast', index_col=0)

# Sumar las ventas por ítem (suma a lo largo de las filas para cada columna)
sales_sum = df_forecast.sum(axis=0)

# Leer la hoja 'Precio_Costo' con la primera columna como índice (ítems)
df_prices = pd.read_excel(url, sheet_name='Precio_Costo', index_col=0)

# Convertir los precios de formato con coma a punto decimal y a float
df_prices['PRECIO'] = df_prices['PRECIO'].astype(str).str.replace(',', '.').astype(float)

# Calcular el ingreso (revenue) para cada ítem: ventas sumadas * precio
revenues = {}
for item in sales_sum.index:
    if item in df_prices.index:
        revenues[item] = sales_sum[item] * df_prices.loc[item, 'PRECIO']

# Convertir a DataFrame para facilitar el manejo
df_abc = pd.DataFrame(list(revenues.items()), columns=['Item', 'Revenue'])
df_abc['Total_Quantity'] = df_abc['Item'].map(sales_sum)  # Añadir cantidad total
df_abc = df_abc.sort_values(by='Revenue', ascending=False).reset_index(drop=True)

# Calcular el porcentaje acumulativo
df_abc['CumRevenue'] = df_abc['Revenue'].cumsum()
df_abc['CumPerc'] = df_abc['CumRevenue'] / df_abc['Revenue'].sum()

# Asignar clases ABC
def assign_abc(cum_perc):
    if cum_perc <= 0.6:
        return 'A'
    elif cum_perc <= 0.8:
        return 'B'
    else:
        return 'C'

df_abc['ABC_Class'] = df_abc['CumPerc'].apply(assign_abc)

# --- Clasificación XYZ ---
# Leer la hoja 'Ventas' con la primera columna como índice (meses)
df_sales = pd.read_excel(url, sheet_name='Ventas', index_col=0)

# Calcular el coeficiente de variación para cada ítem usando los últimos 16 datos
cvs = {}
for item in df_sales.columns:
    series = df_sales[item].dropna().tail(16)
    if len(series) > 0:
        mean = np.mean(series)
        std = np.std(series)
        cv = std / mean if mean != 0 else 0
        cvs[item] = cv

# Crear DataFrame con CV
df_xyz = pd.DataFrame.from_dict(cvs, orient='index', columns=['CV']).reset_index().rename(columns={'index': 'Item'})

# Calcular percentiles
cv_values = df_xyz['CV'].values
p33 = np.percentile(cv_values, 33)
p67 = np.percentile(cv_values, 67)

# Asignar clases XYZ
def assign_xyz(cv):
    if cv <= p33:
        return 'X'
    elif cv <= p67:
        return 'Y'
    else:
        return 'Z'

df_xyz['XYZ_Class'] = df_xyz['CV'].apply(assign_xyz)

# --- Combinar los resultados ---
# Unir los DataFrames ABC y XYZ
df_combined = pd.merge(
    df_abc[['Item', 'Revenue', 'Total_Quantity', 'ABC_Class']],
    df_xyz[['Item', 'CV', 'XYZ_Class']],
    on='Item',
    how='outer'
)

# Reemplazar NaN con '-' para ítems que no estén en ambas clasificaciones
df_combined['ABC_Class'] = df_combined['ABC_Class'].fillna('-')
df_combined['XYZ_Class'] = df_combined['XYZ_Class'].fillna('-')
df_combined['Revenue'] = df_combined['Revenue'].fillna(0)
df_combined['Total_Quantity'] = df_combined['Total_Quantity'].fillna(0)
df_combined['CV'] = df_combined['CV'].fillna(0)

# Ordenar por Revenue descendente
df_combined = df_combined.sort_values(by='Revenue', ascending=False).reset_index(drop=True)

# Imprimir los resultados
print("Clasificación combinada ABC y XYZ de los ítems:")
print(df_combined[['Item', 'Revenue', 'Total_Quantity', 'ABC_Class', 'XYZ_Class']])

# Guardar los resultados en un archivo Excel
df_combined.to_excel('ABC_XYZ_Classification_Result.xlsx', index=False)
print("\nResultados guardados en 'ABC_XYZ_Classification_Result.xlsx'")

Clasificación combinada ABC y XYZ de los ítems:
       Item    Revenue  Total_Quantity ABC_Class XYZ_Class
0    Item_6  261897.23            2749         A         X
1   Item_39  232804.00            2860         A         X
2   Item_49  218780.48            2227         A         Y
3    Item_7  218632.77            2511         A         Y
4    Item_4  208327.68            2816         A         X
5    Item_5  202872.11            2701         A         Z
6   Item_38  179744.20            2188         A         Y
7   Item_27  173868.10            2086         A         Y
8   Item_20  169890.00            2625         A         X
9   Item_13  169719.71            2881         A         Y
10  Item_10  167416.86            2886         A         X
11  Item_22  166969.60            2240         A         Y
12  Item_24  165343.68            2592         A         X
13   Item_1  150256.08            2808         A         X
14  Item_46  138867.32            3011         A         X
15   Ite